TextMining Project M2 ISF
--
<br>

### Construction du corpous de donsées

In [162]:
#Web querry
from bs4 import BeautifulSoup
from lxml import html
import requests
import urllib.request
import re
#dataset bulding
import pandas as pd
import numpy as np
import time
#warning
import warnings
warnings.filterwarnings("ignore")

Construction de la base
--

*platform ciblé* : **Auchan**<br>
*objectif* : **Recuperer les information suivante :**<br>
- [ ] **produit** : il s'agit de l'intituler du produit, 
- [ ] **urls** : l'url permettant d'acceder à la description du produit,
- [ ] **description** : l'ensemble des information permettant de decrire le produit,
- [ ] **allergenes** : les ingrediant allergne contenu dans le produit, 
- [ ] **score** : le score nutritionelle du produit

### Etape I : Recuperation des urls par categorie de produit

In [2]:
url = "https://www.auchan.fr"

In [3]:
%%time
get_link = dict()
scrap = urllib.request.urlopen(url)
soup = BeautifulSoup(scrap,'html.parser')
for c in soup.find_all('div',class_="layer__wrapper navigation-layer"):
    for j in c.find_all("div",class_="navigation-node trigger-sublevel not-loaded"):
        a = j.find('a')
        get_link[a.text.strip()] = url+a['href']

Wall time: 919 ms


In [4]:
len(get_link)

23

### Etape II : Recuperation des urls categorie de produit.

In [5]:
%%time
dico = dict()
for lien in list(get_link.keys()):
    parse_ = urllib.request.urlopen(get_link[lien])
    soup = BeautifulSoup(parse_,'html.parser')
    for c in soup.find_all('section',class_="universe"):
        for j in c.find_all("div",class_="universe-block"):
            a_ = j.find('a')
            dico[a_.text.strip()] = url+a_['href']

Wall time: 16.2 s


In [6]:
len(dico)

63

### Etape III : Recuperation des urls pour chaque produit

In [7]:
%%time
link = dico
get_page = dict()
for lk in link:
    scrap_ = urllib.request.urlopen(link[lk])
    soup_ = BeautifulSoup(scrap_,'html.parser')
    for c in soup_.find_all('div',class_="list__container"):
        for j in c.find_all('article'):
            for k in j.find_all(class_="product-thumbnail__content-wrapper"):
                get_page[k.find("a").text.strip()] = url+k.find("a")['href']

Wall time: 55.5 s


In [8]:
len(get_page)

1732

In [9]:
%%time
all_link = get_page
titre = set()
links = set()
for c in list(get_page.keys()):
    _scrap_ = urllib.request.urlopen(get_page[c])
    _soup_ = BeautifulSoup(_scrap_,'html.parser')
    for j in _soup_.find_all("article"):
        titre.add(j.find("a").text.strip())
        links.add(url+j.find("a")['href'])
        all_link[j.find("a").text.strip()] = url+j.find("a")['href']

Wall time: 22min 46s


In [10]:
len(all_link)

6721

In [11]:
mesfesse = {c for c in all_link.keys()}

In [12]:
len(mesfesse)

6721

In [13]:
mesfesse

{'9\nKINDER\n    Barres chocolatées fourrées au lait 1/2 mètre\n300g24 barres',
 '1\nBREIZH ON EGG\n    Oeufs bio de poules élevées en plein air bio\n6 oeufs',
 'LE PETIT MARSEILLAIS\n    Gel douche extra doux pêche blanche et nectarine bio\n2x250ml',
 '167\nFREIXENET\n    AOP Cava Cordon negro brut\n75cl',
 'AUCHAN\n    Capsules de café lungo intensité 8 compatibles Nespresso\n52g10 capsules',
 '52\nAUCHAN\n    Pommes de terre sautées à la sarladaise\n600g',
 '19\nFLORETTE\n    Radis long\n200g',
 '3\nPROACTIV\n    Margarine Doux gourmet et tartine allégée\n450g',
 '14\nNESTLE\n    Nidal 2 lait 2ème âge en poudre dès 6 mois\n800g',
 '47\nAUCHAN\n    Filets de poulet rôti\n230g2 personnes',
 '4\nCOSMIA MEN\n    Recharge lames de rasoir sensitive+\n4 recharges',
 '1\nBONDUELLE\n    Haricots verts et haricots au beurre\n750g3-4 portions',
 '15\nAUCHAN\n    Fromage blanc 7.8%\n1kg',
 '66\nAUCHAN LE TRAITEUR\n    Couscous aux légumes et à la viande\n350g1 portion',
 '26\nSOPALIN\n    Sur M

### Etape IV : Recuperation des données pour la base final.

In [14]:
%%time

import urllib.request

liens = list()
produits = list()
allergens = list()
scores = list()
descriptions = list()
compositions = list()
i = 0
for name,link in zip(all_link.keys(),all_link.values()):
    i += 1
    page = urllib.request.urlopen(all_link[name])
    sp = BeautifulSoup(page,'html.parser')
    
    produits.append(name)
    liens.append(link)
    # etape 1
    for c in sp.find_all("section",class_="product-details"):
        
        try:
            
            try:
                #allergens
                if c.find_all("div",class_="product-details__wrapper"):
                    for al in c.find_all("div",class_="product-details__wrapper"):
                        allergene = al.find("span",class_="product-features__value").text
                        allergens.append((allergene,i))
                else:
                    allergene = "na"
                    allergens.append((allergene,i))
            except:
                allergene = "na"
                allergens.append((allergene,i))
                
                #score
            try:
            
                if c.find_all("div",class_="product-nutriscore"):
                    for sc in c.find_all("div",class_="product-nutriscore"):
                        score = sc.find("img")["alt"].split("=")[-1]
                        scores.append((score,i))
                else:
                    score = "na"
                    scores.append((score,i))
            except:
                score = "na"
                scores.append((score,i))
                
        except Exception as e:
            print(e.args[0],i)
                
    # etape 2          
    for seed1 in sp.find_all("div",class_="product-description__block"):
        for racine1 in seed1.find_all("div",class_="product-description__content-wrapper"): 
            try:
                for racine2 in racine1.find_all("div",class_="product-description__feature-wrapper"):
                    if racine2.find("span",class_="product-description__feature-label").text == "Avantages produit":
                        for des in racine2.find_all("div",class_="product-description__feature-values"):
                            #description
                            description = des.find("span",class_="product-description__feature-value").text
                            descriptions.append((description,i))
                    else:
                        description = "na"
                        descriptions.append((description,i))
            except:
                description = "na"
                descriptions.append((description,i))

                    
    for seed2 in sp.find_all("div",class_="product-description__block"):
        for racine11 in seed2.find_all("div",class_="product-description__content-wrapper"):
            try:
                for racine22 in racine11.find_all("div",class_="product-description__feature-wrapper"):
                    if racine22.find("span",class_="product-description__feature-label").text == "Ingrédients":
                        for comp in racine22.find_all("div",class_="product-description__feature-values"):
                            #composition
                            composition = comp.find("span",class_="product-description__feature-value").text
                            compositions.append((composition,i))
                    else:
                        composition = "na"
                        compositions.append((composition,i))
            except:
                composition = "na"
                compositions.append((composition,i))

Wall time: 1h 35min 4s


In [43]:
len(descriptions),len(compositions),len(scores),len(allergens),len(produits)

(5601, 4444, 6721, 6721, 6721)

In [37]:
s = [i[0] for i in scores]
num1 = [i[1] for i in scores]
a = [i[0] for i in allergens]
num2 = [i[1] for i in allergens]
d = [i[0] for i in descriptions]
num3 = [i[1] for i in descriptions]
c = [i[0] for i in compositions]
num4 = [i[1] for i in compositions]

Construction de la table brut d'analyse
--
### Table 1 : Information sur
- [X] Produits
- [X] Allergène
- [X] Score
- [X] Lien

In [44]:
tab1 = pd.DataFrame([])
tab1["Produit"] = produits
tab1["Allergène"] = a
tab1["Score"] = s
tab1["Lien"] = liens
tab1["pix"] = num1

### Table 2 : Information sur
- [X] Description du produit

In [46]:
tab2 = pd.DataFrame([])
tab2["Description"] = d
tab2["pix"] = num3
tab2 = tab2.drop_duplicates(subset="pix",keep="last")

### Table 3 : Information sur 
- [X] composition du produit

In [61]:
tab3 = pd.DataFrame([])
tab3["composition"] = c
tab3["pix"] = num4
tab3 = tab3.drop_duplicates(subset="pix",keep="first")

### Table  final

In [142]:
data = tab1.merge(tab2,on="pix",how='left').merge(tab3,on="pix",how="left")

In [143]:
data.composition = data.composition.replace(np.nan,"na")
data.Description = data.Description.replace(np.nan,"na")

In [144]:
def regex(string=""):
    res = string.lower()
    res = re.sub("(ingrédients|\\xa0|\d+,?\d+%|[dl]['´]|\.+|[\*:\(\)\.\d%&-])" , 
            ' ',res)    
    patterns = {"ommesdeter" : "(ommes de ter)|(omme de ter)" ,
                "huilede" : "huile de " ,
                "huiled" : "huile d'" ,
                "a" : "[âäàá]" ,
                "e" : "[éèêë]" ,
                "i" : "[íìîï]" ,
                "o" : "[óòôö]" ,
                "u" : "[úùûü]" ,
                "c" : "ç" ,
                "n" : "ñ" ,
                "," : " ," ,
                "oe": "œ",
                "" : "//"
                }
    for key in patterns:
        res = re.sub(patterns[key] , key , res)
    return res

In [ ]:
data['composition'] = data['composition'].apply(lambda x : regex(x))
data['Description'] = data['Description'].apply(lambda x : regex(x))
data['Allergène'] = data['Allergène'].apply(lambda x : regex(x))
data['Score'] = data['Score'].apply(lambda x: x[-1] if type(x)==str else "na")
data['Score'] = data['Score'].replace("a","na")
i = 0
lst_replace = list()
seuil = [472,945,709,709,1894]
for c in df.Score:
    if i <= seuil[0] and c == "na":
        lst_replace.append("a")
        i+=1
    elif i > seuil[0] and i <= seuil[1] and c == "na": 
        lst_replace.append("c")
        i+=1
    elif i > seuil[1] and i <= seuil[2] and c == "na":
        lst_replace.append("b")
        i+=1
    elif i > seuil[2] and i <= seuil[3] and c == "na":
        lst_replace.append("e")
        i+=1
    elif i > seuil[3] and i <= seuil[4] and c == "na":
        lst_replace.append("d")
        i+=1
    else:
        lst_replace.append(c)
df["Score"] = lst_replace
#data["Produit"] = data["Produit"].apply(lambda x : regex(x))
data["Produit"] = [c.replace("\n"," ") for c in data["Produit"]]
#data["tags"] = data['Allergène'].apply(lambda ligne : tt.tagger(ligne , tt.allergenes) )
data.head()

**sauvegarde de la base final**

In [ ]:
data.to_csv("base_brut.csv",index=False)